In [29]:
import re

import pandas as pd
import geopandas as gpd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib

# Elterngeld: Basisdaten von destatis

Hier führen wir einige Grunddaten ein, die bei destatis verfügbar sind.

## Geburten

Die Zahl der Geburten dient als Grundlage, ohne die Schwankungen beim Elterngeldbezug nicht zu verstehen sind.
Da die Spanne zwischen Bremen und NRW sehr groß ist und wir uns mehr für Unterschiede in den relativen Geburtenraten zwischen Ländern und über die Jahre hinweg interessieren, skalieren wir die Geburtenrate auf "Geburten je 100.000 Einwohner:innen".
Die Tabelle enthält auch das Geschlecht des Kindes, das ignorieren wir hier.

In [30]:
gb = pd.read_parquet("../data/processed/geburten.parquet")
gb = gb.groupby(["jahr", "land"]).geburten.sum().to_frame().reset_index()

ewz = pd.read_parquet("../data/processed/ewz.parquet")

gb = pd.merge(gb, ewz, on=["jahr", "land"])
gb["geburten_pro_1000"] = gb.geburten / gb.ewz * 1000
# gb.land = gb.land.str.replace("-", "-<br>")

fig = go.Figure()

for land, grp in gb.groupby("land"):
    fig.add_trace(
        go.Scatter(
            x=grp.jahr,
            y=grp.geburten_pro_1000,
            mode="markers+lines",
            name=land
        ),
    )

fig.update_layout(
    width=1000,
    height=800,
    margin=dict(t=20, r=20, b=20, l=20)
)

fig.show()

Interessant ist wie 2021/22 in den alten Bundesländern die Zahl wieder steigt, aber in den neuen sich der Abwärtstrend nicht umkehrt.
Möglicherweise gibt es Geschlechterunterschiede in Wanderungsbewegungen?
Die Geburtenrate pro Mutter wäre interessant.

## Empfangende von Elterngeld


In [31]:
eg = pd.read_parquet("../data/processed/eg_empf.parquet")

eg = eg.loc[eg.art.eq("Insgesamt")]
eg = eg.loc[eg.fm.ne("Insgesamt")]

eg = eg.groupby(["jahr", "land", "fm", "art"]).sum().drop("quartal", axis=1).reset_index()

eg["grp_display"] = eg.apply(
    lambda row: f"{'Mütter' if row.fm=='weiblich' else 'Väter'} in {row.land}",
    axis=1
)

eg = eg.sort_values(by=["jahr", "land", "fm"], ascending=[True, True, False]).reset_index(drop=True)
land_clr = {
    'Schleswig-Holstein': '#1f77b4',
    'Hamburg': '#ff7f0e',
    'Niedersachsen': '#2ca02c',
    'Bremen': '#d62728',
    'Nordrhein-Westfalen': '#9467bd',
    'Hessen': '#8c564b',
    'Rheinland-Pfalz': '#e377c2',
    'Baden-Württemberg': '#7f7f7f',
    'Bayern': '#bcbd22',
    'Saarland': '#17becf',
    'Berlin': '#aec7e8',
    'Brandenburg': '#ffbb78',
    'Mecklenburg-Vorpommern': '#98df8a',
    'Sachsen': '#ff9896',
    'Sachsen-Anhalt': '#c5b0d5',
    'Thüringen': '#c49c94',
}

fm_line = {
    "weiblich": "solid",
    "männlich": "dash",
}

fig = go.Figure()

for var, grp in eg.groupby(["land", "fm"]):
    fig.add_trace(
        go.Scatter(
            x=grp.jahr,
            y=grp.pers,
            mode="markers+lines",
            line=dict(
                dash=fm_line[var[1]],
                color=land_clr[var[0]]
            ),
            name=var[0],
            showlegend=var[1]=="weiblich",
            legendgroup=var[0],
            visible=True if var[0] == "Berlin" else "legendonly",
            customdata=grp.grp_display.values,
            hovertemplate="%{customdata}: %{y:f}<extra></extra>"
        )
    )

fig.update_layout(
    hovermode="x unified",
    width=1000,
    height=550,
    margin=dict(t=50, r=20, b=20, l=20),
    yaxis=dict(
        range=[0, eg.pers.max()],
        tickformat=".0f"
    ),
    title="Empfänger:innen von Elterngeld"
)

fig.show()

# Empfänger:innen von Elterngeld in Prozent Anteil an Geburten

In [32]:
egb = pd.merge(
    eg,
    gb[["jahr", "land", "geburten"]],
    on=["jahr", "land"],
    how="left"
)
egb["eg_rate"] = egb.pers / egb.geburten * 100

fig = go.Figure()

for var, grp in egb.groupby(["land", "fm"]):
    fig.add_trace(
        go.Scatter(
            x=grp.jahr,
            y=grp.eg_rate,
            mode="markers+lines",
            line=dict(
                dash=fm_line[var[1]],
                color=land_clr[var[0]]
            ),
            name=var[0],
            showlegend=var[1]=="weiblich",
            legendgroup=var[0],
            visible=True if var[0] == "Berlin" else "legendonly",
            customdata=grp.grp_display.values,
            hovertemplate="%{customdata}: bei %{y:.1f}% der geborenen Kinder<extra></extra>"
        )
    )

fig.update_layout(
    hovermode="x unified",
    width=1000,
    height=800,
    margin=dict(t=20, r=20, b=20, l=20),
    yaxis=dict(range=[0, egb.eg_rate.max()])
)

fig.show()

# Dauer des Bezugs ~ Steuerkraft

## Teil 1: Dauer des Bezugs als Karte

Daten liegen in Langform vor. Wir formen hin zu einer Zeile pro Kreis und Jahr.

In [28]:
df_dauer_krs = pd.read_parquet("../data/processed/eg_dauer.parquet")

# viele heute eingestellte Kreise mit fehlenden Daten; entfernen:
df_dauer_krs = df_dauer_krs.dropna(subset="monate")

df_dauer_krs

,jahr,krs,fm,egplus,monate
18,2016,"Ahrweiler, Landkreis",Insgesamt,Insgesamt,12.0
19,2016,"Ahrweiler, Landkreis",Insgesamt,Mit Elterngeld Plus,19.5
20,2016,"Ahrweiler, Landkreis",Insgesamt,Ohne Elterngeld Plus,9.7
21,2016,"Ahrweiler, Landkreis",männlich,Insgesamt,3.3
22,2016,"Ahrweiler, Landkreis",männlich,Mit Elterngeld Plus,7.4
...,...,...,...,...,...
34247,2023,"Zwickau, Landkreis",Insgesamt,Ohne Elterngeld Plus,8.0
34249,2023,"Zwickau, Landkreis",männlich,Mit Elterngeld Plus,6.5
34251,2023,"Zwickau, Landkreis",weiblich,Insgesamt,14.5
34252,2023,"Zwickau, Landkreis",weiblich,Mit Elterngeld Plus,17.8


In [14]:
# Geschlecht und EG-Plus zu Spalten machen:
dd_piv = df_dauer_krs.pivot(index=["jahr", "krs"], columns=["fm", "egplus"], values="monate").reset_index()
dd_piv

fm      jahr                                   krs Insgesamt  \
egplus                                             Insgesamt   
0       2016                  Ahrweiler, Landkreis      12.0   
1       2016          Aichach-Friedberg, Landkreis      10.3   
2       2016                       Alb-Donau-Kreis      10.4   
3       2016           Altenburger Land, Landkreis      11.9   
4       2016  Altenkirchen (Westerwald), Landkreis      13.1   
...      ...                                   ...       ...   
3203    2023                   Würzburg, Landkreis      10.9   
3204    2023            Würzburg, kreisfreie Stadt      10.7   
3205    2023                       Zollernalbkreis      12.2   
3206    2023         Zweibrücken, kreisfreie Stadt      13.4   
3207    2023                    Zwickau, Landkreis      11.1   

fm                                               männlich                      \
egplus Mit Elterngeld Plus Ohne Elterngeld Plus Insgesamt Mit Elterngeld Plus   
0                     19.5                  9.7       3.3                 7.4   
1                     18.5                  9.0       3.2                 9.0   
2                     18.8                  8.9       2.8                 7.9   
3                     18.6                  9.3       3.2                 8.0   
4                     20.9                  9.7       3.4                10.4   
...                    ...                  ...       ...                 ...   
3203                  17.5                  8.0       3.2                 8.1   
3204                  16.5                  8.5       4.0                 8.4   
3205                  18.9                  8.1      <NA>                 7.2   
3206                  19.9                  9.3       3.2                 5.7   
3207                  16.5                  8.0      <NA>                 6.5   

fm                           weiblich                                           
egplus Ohne Elterngeld Plus Insgesamt Mit Elterngeld Plus Ohne Elterngeld Plus  
0                       2.7      14.0                20.8                 11.7  
1                       2.8      13.0                20.1                 11.6  
2                       2.5      13.3                20.3                 11.7  
3                       2.9      14.3                19.1                 11.8  
4                       2.7      15.1                21.4                 11.8  
...                     ...       ...                 ...                  ...  
3203                   <NA>      14.5                19.2                 11.7  
3204                   <NA>      13.7                18.6                 11.4  
3205                   <NA>      15.7                20.1                 11.6  
3206                   <NA>      15.4                20.5                 11.4  
3207                   <NA>      14.5                17.8                 11.7  

[3208 rows x 11 columns]

Hier wird etwas klarer, was für Daten fehlen:

In [15]:
for jahr, grp in dd_piv.groupby("jahr"):
    print(jahr, end=":\n")
    print(grp.isna().sum().to_frame(f"Gesamt: {len(grp)} Kreise"), end="\n-----\n")

2016:
                                Gesamt: 401 Kreise
fm        egplus                                  
jahr                                             0
krs                                              0
Insgesamt Insgesamt                              0
          Mit Elterngeld Plus                    0
          Ohne Elterngeld Plus                   0
männlich  Insgesamt                              0
          Mit Elterngeld Plus                    0
          Ohne Elterngeld Plus                   0
weiblich  Insgesamt                              0
          Mit Elterngeld Plus                    0
          Ohne Elterngeld Plus                   0
-----
2017:
                                Gesamt: 401 Kreise
fm        egplus                                  
jahr                                             0
krs                                              0
Insgesamt Insgesamt                              0
          Mit Elterngeld Plus                    0
          Ohn

Ab 2019 fehlen die Daten zu den Vätern in beträchtlichem Maße - ein Viertel der Gesamtwerte und 75% der Werte ohne EG-Plus.
Es könnte daher lohnen, einmal auf Männer insgesamt zu gucken (auch wenn da bis zu 25% fehlen) und dann auf Männer mit EG-Plus.

Wir matchen die EG-Daten auf Kreisebene mit Geodaten aus der VG250 des Bundesamts für Kartografie und Geodäsie. Warum? Weil wir nur so die Benennungen der Kreise im EG-Datensatz mit tatsächlichen Entitäten verbinden können, die eindeutig identifiziert sind und für die es z.B. Geometriedaten gibt. Die Benennung der Kreise im EG-Datensatz ist zudem etwas unsauber und das wollen wir korrigieren.

In [ ]:
# Geodaten laden:
vg_krs = gpd.read_file("../data/raw/vg250-ew_12-31.utm32s.shape.ebenen/vg250-ew_ebenen_1231/VG250_KRS.shp").to_crs(epsg=4326)

# Seegebiete ausschließen:
vg_krs = vg_krs.loc[vg_krs.GF.ne(2)]

# schönere Spaltennamen:
columns = {
    "AGS_0": "ags",
    "GEN": "gen",
    "BEZ": "bez",
    "EWZ": "ewz",
    "geometry": "geom",
}
vg_krs = vg_krs.filter(columns).rename(columns, axis=1)

# Unser erster Match-Versuch nutzt die Tatsache, dass im EG-Datensatz die Kreise benannt sind nach der Form:
# "<NAME>, <BEZEICHNUNG>".
# Wir konstruieren im gdf eine neue Spalte aus "gen" & "bez", die das reproduziert.

# Dafür müssen wir die Großschreibung korrigieren:
vg_krs.loc[vg_krs.bez.eq("Kreisfreie Stadt"), "bez"] = "kreisfreie Stadt"

vg_krs["krs"] = vg_krs.gen + ", " + vg_krs.bez
vg_krs

Wir füllen `df_dauer_matched` nach und nach mit den Matches zwischen `df_dauer_krs` und der `vg_krs`.

Erster Join: die `krs`-Spalte im EG-Datensatz entspricht der Form "@gen, @bez"; das machen wir uns zunutze:

In [ ]:
df_dauer_matched = pd.merge(df_dauer_krs, vg_krs, on="krs")

# Nichtmatches:
df_dauer_mismatch = df_dauer_krs.query('~krs.isin(@df_dauer_matched.krs)')
df_dauer_mismatch

Zweiter Join: die Bezeichnung des Kreises (Stadtkreis, Landkreis, kreisfreie Stadt, ...) ist bei einigen Orten Teil von `gen`; wir matchen die bisherigen Überbleibsel, die deswegen nicht gematcht haben und die erfolgreichen Matches wandern in df_dauer_matched:

In [ ]:
df_dauer_matched = pd.concat([
    df_dauer_matched,
    pd.merge(df_dauer_mismatch, vg_krs.drop("krs", axis=1), left_on="krs", right_on="gen")
])
df_dauer_mismatch = df_dauer_krs.query('~krs.isin(@df_dauer_matched.krs)')
df_dauer_mismatch

Dritter & vierter Join: Einige Kreise, die im EG-Datensatz "Landkreis" oder "kreisfreie Stadt" heißen, heißen in der vg250 "Kreis" oder "Stadtkreis":

In [ ]:
vg_krs.krs = vg_krs.krs.str.replace("Kreis", "Landkreis")
df_dauer_matched = pd.concat([
    df_dauer_matched,
    pd.merge(df_dauer_mismatch, vg_krs, on="krs")
])
df_dauer_mismatch = df_dauer_krs.query('~krs.isin(@df_dauer_matched.krs)')
df_dauer_mismatch

In [ ]:
vg_krs.krs = vg_krs.krs.str.replace("Stadtkreis", "kreisfreie Stadt")
df_dauer_matched = pd.concat([
    df_dauer_matched,
    pd.merge(df_dauer_mismatch, vg_krs, on="krs")
])
df_dauer_mismatch = df_dauer_krs.query('~krs.isin(@df_dauer_matched.krs)')
df_dauer_mismatch

Und zuletzt gibt es einfach einige Totalverluste, vor allem weil im BKG seit einiger Zeit eine neue Nomenklatur für Präpositionen im Namen gilt: abgekürzt und ohne Leerzeichen bei mehreren Wörtern. Plus, wir kürzen "Oberpfalz" und "Oldenburg" ab.

In [ ]:
translatedict = {
    "Dillingen a.d. Donau, Landkreis": 'Dillingen an der Donau, Landkreis',
    "Eisenach, kreisfreie Stadt": 'Eisenach, kreisfreie Stadt (bis 30.06.2021)',
    "Mühldorf a. Inn, Landkreis": 'Mühldorf am Inn, Landkreis',
    "Neumarkt i.d. OPf., Landkreis": 'Neumarkt in der Oberpfalz, Landkreis',
    "Neustadt a.d. Aisch-Bad Windsheim, Landkreis": 'Neustadt an der Aisch-Bad Windsheim, Landkreis',
    "Neustadt a.d. Waldnaab, Landkreis": 'Neustadt an der Waldnaab, Landkreis',
    "Oldenburg (Oldb), kreisfreie Stadt": 'Oldenburg (Oldenburg), kreisfreie Stadt',
    "Pfaffenhofen a.d. Ilm, Landkreis": 'Pfaffenhofen an der Ilm, Landkreis',
    "St. Wendel, Landkreis": 'Sankt Wendel, Landkreis',
    "Weiden i.d. OPf., kreisfreie Stadt": 'Weiden in der Oberpfalz, kreisfreie Stadt',
    "Wunsiedel i. Fichtelgebirge, Landkreis": 'Wunsiedel im Fichtelgebirge, Landkreis'
}

vg_krs.krs = vg_krs.krs.replace(translatedict)

df_dauer_matched = pd.concat([
    df_dauer_matched,
    pd.merge(df_dauer_mismatch, vg_krs, on="krs")
])
df_dauer_mismatch = df_dauer_krs.query('~krs.isin(@df_dauer_matched.krs)')
df_dauer_mismatch

Einzig übrig ist nur noch Eisenach, das 2021 aufgehört hat als Kreis zu existieren. Das können wir dann ignorieren.

In [ ]:
df = gpd.GeoDataFrame(df_dauer_matched, geometry="geom")
df

In [ ]:
fig, axs = plt.subplots(
    ncols=3,
    nrows=1,
    sharex=True, sharey=True,
)

fig.set_size_inches(20, 7.5)

z_var = "monate"
xfac_var = "fm"


# globales Minimum und Maximum bei der Farb-Variable:
z_min = df[z_var].min()
z_max = df[z_var].max()

# der Teil hier ist dann als interaktiver Teil realisiert:
df_plot = df.query(
    'egplus == "Mit Elterngeld Plus"'
    'and jahr == 2023'
)

facet_var = "fm"
facets = df_plot[facet_var].unique()

cols = np.array([0, 1, 2])
subpltcoord = cols

for xy, facet in zip(subpltcoord, facets):

    df_facet = df_plot.loc[df_plot[facet_var].eq(facet)]

    x = xy
    df_facet.plot(
        ax=axs[x],
        column=z_var,
        vmin=z_min,
        vmax=z_max,
    )

    axs[x].set_title(facet)

for ax in axs:
    ax.axis("off")

cbar = plt.colorbar(axs[0].collections[0], ax=axs.ravel().tolist(), shrink=.8)
cbar.set_label(z_var)

In [ ]:
n = matplotlib.colors.Normalize(0, 4)
n(.9)

In [ ]:
fig = go.Figure()

df_plot = df_dauer_matched.query(
    '(jahr == 2023)'
    'and (fm == "weiblich")'
    'and (egplus == "Insgesamt")'
)

In [ ]:
df.jahr.nunique()

# Wunschliste

- User können in der App entscheiden, welche Variable als Facette dargestellt wird
- Hover
- Zoom